# Attribute Descriptors

A descriptor is a class that implements a dynamic protocol consisting of the __get__,
__set__, and __delete__ methods.

### Descriptor Example: Attribute Validation

### LineItem Take #3: A Simple Descriptor
As we said in the introduction, a class implementing a __get__, a __set__, or a
__delete__ method is a descriptor. You use a descriptor by declaring instances of it
as class attributes of another class.

In [2]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
    def __get__(self, instance, owner):
        return instance.__dict__[self.storage_name]

In [3]:
class House:
    rooms = Quantity('number_of_rooms')

In [4]:
def __get__(self, instance, owner):
    if instance is None:
        return self
    else:
        return instance.__dict__[self.storage_name]

In [5]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

LineItem Take #4: Automatic Naming of Storage Attributes

In [12]:
class Quantity2:
    def __set_name__(self, owner, name):
        self.storage_name = name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

# no __get__ needed
class LineItem2:
    weight = Quantity2()
    price = Quantity2()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price


In [18]:
obj1 = LineItem2('abi', 10, 10)

### LineItem Take #5: A New Descriptor Type

Example 23-5. model_v5.py: the Validated ABC

In [19]:
import abc
class Validated(abc.ABC):
    def __set_name__(self, owner, name):
        self.storage_name = name
    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)
        instance.__dict__[self.storage_name] = value
    @abc.abstractmethod
    def validate(self, name, value):
        """return validated value or raise ValueError"""

In [22]:
class VQuantity(Validated):
    """a number greater than zero"""
    def validate(self, name, value):
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, name, value):
        value = value.strip()
        if not value:
            raise ValueError(f'{name} cannot be blank')
        return value

In [24]:
class LineItem3:
    description = NonBlank()
    weight = VQuantity()
    price = VQuantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [33]:
obh = LineItem3('abi', 10, 3)

In [34]:
obh.subtotal()

30